Create and Test Team Ratings for 2022 Season - Round by Round

In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

In [2]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
27846,202323_WesternBulldogs_WestCoast,Western Bulldogs,Oskar Baker,202323,Oskar_Baker,MIDFIELDER,CD_I1008855,25,184,87,13,RIGHT,QLD,2017.0,2019.0,Wilston Grange (Qld)/Aspley (NEAFL)/Melbourne,48.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Half Forward Flank Right,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,7.9598,-0.3226,0.0167,0.3991,0.7006,-0.3015,2.8523,3.8194,-0.9671,-0.3267,0.0713,1.0483,3.4625,Oskar Baker_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27847,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rhylee West,202323,Rhylee_West,MEDIUM_FORWARD,CD_I1006127,23,183,82,14,RIGHT,VIC,2018.0,2019.0,Strathmore (Vic)/St Kevin&apos;s College (Vic)...,26.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,9.7706,0.1389,0.0000,0.5694,0.6452,-0.0758,7.0749,7.4761,-0.4012,0.4868,0.0043,0.9269,6.2204,Rhylee West_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27848,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rory Lobb,202323,Rory_Lobb,KEY_FORWARD,CD_I990740,30,207,106,7,RIGHT,WA,2013.0,2014.0,Bassendean (WA)/Swan Districts (WAFL)/GWS/Frem...,29.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Left,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,17.1166,-0.1439,-0.0025,1.7295,1.8518,-0.1224,11.1693,12.7523,-1.5830,0.4878,-0.0002,1.1501,10.5511,Rory Lobb_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27849,202323_WesternBulldogs_WestCoast,Western Bulldogs,Taylor Duryea,

Pre-Season

In [59]:
def get_previous_round_id(round_id):
    
    finals_round_map = {'F1':24,
                        'F2':25,
                        'F3':26,
                        'F4':27}
    
    season = round_id[:4]
    if 'F' in round_id:
        round_num = finals_round_map[round_id[4:]]
    else:
        round_num = round_id[4:]
    
    previous_round_num = int(round_num)-1
    previous_round_num = str(previous_round_num).zfill(2)
    if previous_round_num == 1:
        previous_season = int(season) - 1
    else:
        previous_season = season
    previous_round_id = str(previous_season) + str(previous_round_num)
    
    return previous_round_id

In [67]:
round_id = "2022F1"
season = round_id[:4]
if 'F' in round_id:
    round_num = finals_round_map[round_id[4:]]
else:
    round_num = round_id[4:]

previous_round_num = int(round_num)-1
previous_round_num = str(previous_round_num).zfill(2)
if previous_round_num == 1:
    previous_season = int(season) - 1
else:
    previous_season = season
previous_round_id = str(previous_season) + str(previous_round_num)

In [4]:
def get_score_standard_deviation(data):
    return data.groupby(['Match_ID', 'Team']).sum()['Score'].std()

In [5]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby(['Team']).sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby(['Team']).count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary

In [6]:
def get_teams(match_id):
    
    home_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[1])
    away_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[-1])

    return home_team, away_team

In [7]:
def calculate_match_projections(home_attack_rating, home_defence_rating, away_attack_rating, away_defence_rating):
    
    home_projected_score = (home_attack_rating + away_defence_rating)*0.5
    away_projected_score = (home_defence_rating + away_attack_rating)*0.5
    projected_margin = home_projected_score - away_projected_score
    
    return home_projected_score, away_projected_score, projected_margin

In [8]:
def get_match_ratings(team_summary, home_team, away_team):
    
    match_ratings_dict = {'home_attack_rating':team_summary.loc[home_team]['exp_vaep_value_pergame_F'],
                          'home_defence_rating':team_summary.loc[home_team]['exp_vaep_value_pergame_A'],
                          'away_attack_rating':team_summary.loc[away_team]['exp_vaep_value_pergame_F'],
                          'away_defence_rating':team_summary.loc[away_team]['exp_vaep_value_pergame_A']}
    
    return match_ratings_dict

In [9]:
def calculate_projected_probabilities(home_projected_score, away_projected_score, score_std=25):
    
    projected_margin = home_projected_score - away_projected_score
    projected_margin_distribution = stats.norm(loc = projected_margin, scale = score_std)
    
    away_team_win_probability = projected_margin_distribution.cdf(0)
    draw_probability = projected_margin_distribution.pdf(0)
    home_team_win_probability = 1 - away_team_win_probability - draw_probability
    
    return home_team_win_probability, draw_probability, away_team_win_probability
    

In [10]:
def get_match_prediction_metrics(projected_margin, margin):
    
    mae = abs(projected_margin - margin)
    mse = ((projected_margin)**2 + (margin)**2)
    rmse = ((projected_margin)**2 + (margin)**2)**0.5
    
    return mae, mse, rmse

In [11]:
def get_match_result(data, match_id):
    
    home_team, away_team = get_teams(match_id)
    match_player_stats = data[data['Match_ID'] == match_id]
    match_score = match_player_stats.groupby('Team').sum()[['Score', 'xScore']]
    home_score, home_xscore = match_score.loc[home_team]['Score'], match_score.loc[home_team]['xScore']
    away_score, away_xscore = match_score.loc[away_team]['Score'], match_score.loc[away_team]['xScore']
    margin, xmargin = home_score - away_score, home_xscore - away_xscore
    
    return home_score, away_score, margin

In [12]:
def calculate_match_projection_error(player_stats, match_id, match_ratings_dict, score_std = 25):
    
    home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                             match_ratings_dict['home_defence_rating'], 
                                                                             match_ratings_dict['away_attack_rating'], 
                                                                             match_ratings_dict['away_defence_rating'])
    
    home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
    home_score, away_score, margin = get_match_result(player_stats, match_id)
    
    mae, mse, rmse = get_match_prediction_metrics(projected_margin, margin)
    
    return mae, mse, rmse

In [13]:
def get_latest_rounds(round_id, player_stats, rounds):
    
    all_rounds = sorted(list(set(player_stats['Round_ID'])))
    rating_rounds = all_rounds[all_rounds.index(round_id)-rounds:all_rounds.index(round_id)]
    player_stats_latest = player_stats[player_stats['Round_ID'].isin(rating_rounds)]
    
    return player_stats_latest

In [14]:
def update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae):
    
    previous_round_id = get_previous_round_id(round_id)
    player_stats_latest = get_latest_rounds(round_id, player_stats, rounds)
    team_summary = create_team_summary(player_stats_latest, rounds = rounds)
    team_summary = team_summary[['exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']]

    round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
    round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
    print("Round MAE: {}".format(round_mae[round_id]))
    print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
    return team_summary.sort_values('exp_vaep_value_pergame_diff', ascending = False)

In [15]:
def create_round_match_projections(player_stats, team_summary, round_id):
    
    player_stats_round = player_stats[player_stats['Round_ID'] == round_id]
    match_id_list = list(player_stats_round['Match_ID'].unique())

    match_projection_dict = {}
    for match_id in match_id_list:
        home_team, away_team = get_teams(match_id)
        match_ratings_dict = get_match_ratings(team_summary, home_team, away_team)
        home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                                                    match_ratings_dict['home_defence_rating'], 
                                                                                                    match_ratings_dict['away_attack_rating'], 
                                                                                                    match_ratings_dict['away_defence_rating'])
        home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
        home_score, away_score, margin = get_match_result(player_stats_round, match_id)
        mae, mse, rmse = calculate_match_projection_error(player_stats_round, match_id, match_ratings_dict, score_std = 25)

        match_projection_dict[match_id] = {}
        if home_projected_score > away_projected_score:
            match_projection_dict[match_id]['tip'] = home_team
            match_projection_dict[match_id]['projected_prob'] = home_team_win_probability
        else:
            match_projection_dict[match_id]['tip'] = away_team
            match_projection_dict[match_id]['projected_prob'] = away_team_win_probability
        match_projection_dict[match_id]['projected_margin'] = abs(projected_margin)
        if home_score > away_score:
            match_projection_dict[match_id]['result'] = home_team
        else:
            match_projection_dict[match_id]['result'] = away_team
        match_projection_dict[match_id]['actual_margin'] = abs(margin)   
        match_projection_dict[match_id]['MAE'] = mae
    
    return match_projection_dict
    

In [16]:
score_std = get_score_standard_deviation(player_stats)

In [17]:
round_projection_dict = {}
round_mae = {}
rounds = 10

202201

In [18]:
round_id = '202201'
player_stats_latest = player_stats[player_stats['Season'] == 2021]
team_summary = create_team_summary(player_stats_latest, rounds = rounds)
team_summary = team_summary[['exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']]

round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
print("Round MAE: {}".format(round_mae[round_id]))
print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
team_summary.sort_values('exp_vaep_value_pergame_diff', ascending = False)

Round MAE: 23.52035197400336
Overall MAE: 23.52035197400336


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,123.4076,88.3974,35.0102
Brisbane Lions,135.4402,113.9884,21.4518
Port Adelaide,115.7847,97.4364,18.3483
Sydney,116.0347,101.4422,14.5925
Geelong,108.2664,100.1929,8.0735
Essendon,114.8386,110.3172,4.5214
Richmond,106.1284,104.3544,1.7740
St Kilda,100.4031,99.0970,1.3061
Western Bulldogs,106.3053,105.4237,0.8816
Greater Western Sydney,112.6354,112.6492,-0.0139


In [19]:
round_id = '202202'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 27.726401068808514
Overall MAE: 25.623376521405937


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,125.8118,88.0446,37.7673
Brisbane Lions,134.2051,109.9072,24.2979
Sydney,119.9734,104.2079,15.7655
Port Adelaide,112.6901,98.9718,13.7183
Geelong,109.4059,103.4612,5.9447
St Kilda,102.1498,99.3261,2.8237
Hawthorn,110.0446,107.8808,2.1638
Greater Western Sydney,115.0554,116.0956,-1.0402
Western Bulldogs,105.3058,106.8299,-1.5241
Adelaide,110.3438,113.0838,-2.7399


In [20]:
round_id = '202203'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.44490032573428
Overall MAE: 26.563884456182052


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,125.1826,87.8133,37.3693
Brisbane Lions,129.1236,111.6696,17.4540
Sydney,115.7264,104.3061,11.4203
Port Adelaide,113.5659,104.6049,8.9610
St Kilda,101.9997,94.1166,7.8831
Hawthorn,112.5976,107.1431,5.4545
Geelong,108.7158,104.5373,4.1785
Richmond,107.9190,104.1145,3.8044
Western Bulldogs,107.3850,107.0828,0.3022
Collingwood,106.5877,108.0805,-1.4928


202204

In [21]:
round_id = '202204'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.53766370857302
Overall MAE: 27.057329269279794


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,122.2762,90.4050,31.8712
Brisbane Lions,136.5118,107.3656,29.1462
St Kilda,108.8364,95.4277,13.4087
Port Adelaide,114.6902,106.4009,8.2893
Sydney,112.7667,107.4388,5.3279
Hawthorn,110.2474,105.8159,4.4315
Western Bulldogs,108.7919,107.9863,0.8056
Greater Western Sydney,118.8821,119.7366,-0.8546
Geelong,108.4438,110.1188,-1.6750
Richmond,109.8456,114.2866,-4.4409


202205

In [22]:
round_id = '202205'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.597258902531948
Overall MAE: 27.365315195930226


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,122.3176,90.4845,31.8331
Brisbane Lions,128.9047,110.1383,18.7663
St Kilda,111.5495,97.0796,14.4699
Sydney,117.3378,107.0018,10.3360
Fremantle,103.7744,97.0980,6.6764
Port Adelaide,109.2234,105.3548,3.8686
Geelong,107.1785,106.4692,0.7092
Collingwood,112.8439,112.2399,0.6040
Hawthorn,109.2801,110.5611,-1.2810
Adelaide,113.5601,115.4926,-1.9325


202206

In [23]:
round_id = '202206'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 24.6257061743883
Overall MAE: 26.908713692339905


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,122.4691,90.0269,32.4422
St Kilda,113.2081,92.7274,20.4806
Sydney,119.6761,103.7942,15.8819
Brisbane Lions,125.8105,111.0756,14.7349
Fremantle,108.3221,96.0373,12.2849
Collingwood,114.0936,108.1143,5.9793
Western Bulldogs,114.2921,109.7454,4.5467
Adelaide,114.6619,113.1992,1.4627
Gold Coast,109.5245,109.4663,0.0582
Geelong,110.6388,111.7656,-1.1268


202207

In [24]:
round_id = '202207'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 33.639892974686916
Overall MAE: 27.870310732675193


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,124.5965,85.8894,38.7071
Fremantle,117.7547,96.0683,21.6864
St Kilda,116.2285,97.2051,19.0234
Sydney,122.0146,106.4469,15.5677
Brisbane Lions,125.3999,111.7856,13.6143
Collingwood,114.9515,105.4291,9.5224
Gold Coast,118.8155,112.1976,6.6179
Geelong,116.2979,110.9842,5.3136
Western Bulldogs,116.0308,112.8973,3.1334
Adelaide,113.9635,115.6820,-1.7184


202208

In [25]:
round_id = '202208'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.59158219105568
Overall MAE: 28.335469664972752


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,122.9116,86.2584,36.6532
Fremantle,116.6902,96.2498,20.4404
Brisbane Lions,126.4140,108.4694,17.9445
St Kilda,117.4868,99.9511,17.5358
Collingwood,118.2939,105.5720,12.7220
Sydney,120.1013,110.6300,9.4713
Geelong,116.9446,109.2015,7.7431
Carlton,122.3204,117.5004,4.8200
Western Bulldogs,117.2484,112.9848,4.2636
Richmond,114.5101,111.0113,3.4988


202209

In [26]:
round_id = '202209'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.653987611963277
Overall MAE: 28.481971659082813


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,122.4363,87.2306,35.2057
Fremantle,120.1344,93.7931,26.3413
Brisbane Lions,129.7298,106.3181,23.4116
St Kilda,114.7985,102.2271,12.5715
Collingwood,119.3787,109.2155,10.1632
Carlton,125.3162,115.4519,9.8642
Geelong,117.2311,108.9655,8.2656
Sydney,117.3043,111.4060,5.8983
Richmond,117.0363,113.0063,4.0300
Gold Coast,117.0200,113.6574,3.3626


202210

In [27]:
round_id = '202210'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 19.57648712774609
Overall MAE: 27.591423205949138


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,120.9738,86.5037,34.4701
Fremantle,117.7525,94.5237,23.2288
Brisbane Lions,127.6706,106.1241,21.5464
St Kilda,118.0046,103.0290,14.9755
Carlton,126.9440,113.8307,13.1133
Geelong,121.2482,109.3814,11.8668
Sydney,120.2743,109.8400,10.4343
Richmond,119.2237,111.6708,7.5529
Port Adelaide,112.9358,105.9326,7.0032
Collingwood,116.2583,112.2789,3.9794


202211

In [28]:
round_id = '202211'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.494068521476454
Overall MAE: 27.673481870997076


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,119.1144,85.3909,33.7236
Brisbane Lions,128.9624,110.7134,18.2490
Fremantle,115.3522,97.4144,17.9377
Carlton,129.6215,115.6124,14.0091
Geelong,121.5159,107.7885,13.7274
Richmond,121.2206,109.0507,12.1699
Western Bulldogs,125.4488,113.5035,11.9453
St Kilda,115.6959,104.7312,10.9647
Sydney,121.4116,114.2279,7.1837
Collingwood,116.9755,110.4259,6.5496


202212

In [29]:
round_id = '202212'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 19.43019711802407
Overall MAE: 26.98654147491599


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,114.4917,87.2458,27.2459
Western Bulldogs,133.6208,108.3787,25.2421
Brisbane Lions,129.4528,109.5389,19.9139
Fremantle,113.3112,93.8662,19.4450
St Kilda,115.6543,100.8022,14.8521
Richmond,122.5942,108.6334,13.9607
Geelong,117.2525,106.2738,10.9787
Collingwood,118.3913,109.2465,9.1448
Carlton,127.9607,119.7737,8.1870
Sydney,120.2516,112.6357,7.6159


202213

In [30]:
round_id = '202213'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 16.624086157692336
Overall MAE: 26.189429527437248


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Melbourne,114.0977,88.1332,25.9645
Western Bulldogs,130.6263,107.8989,22.7274
Fremantle,118.0987,96.5243,21.5744
Brisbane Lions,130.1553,110.7478,19.4075
St Kilda,118.0375,103.3421,14.6955
Richmond,121.3128,108.5512,12.7616
Geelong,117.7945,105.8800,11.9145
Carlton,130.1840,119.1818,11.0022
Sydney,121.4522,112.4442,9.0080
Gold Coast,122.4276,114.4331,7.9944


202214

In [31]:
round_id = '202214'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 24.426055191826123
Overall MAE: 26.063474217750734


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Richmond,125.3694,103.8648,21.5045
Western Bulldogs,130.4455,109.8489,20.5966
Melbourne,111.3687,93.0091,18.3596
Fremantle,120.0361,101.9300,18.1061
Geelong,118.6262,103.8596,14.7666
Sydney,124.9081,110.2431,14.6650
Brisbane Lions,124.7612,111.7871,12.9741
Gold Coast,123.8601,112.3901,11.4700
Carlton,128.4711,117.0449,11.4262
Collingwood,118.9895,109.4405,9.5490


202215

In [32]:
round_id = '202215'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 25.19191116347408
Overall MAE: 26.00537001413229


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Western Bulldogs,137.0211,111.6349,25.3862
Richmond,126.0095,104.4315,21.5780
Melbourne,113.1616,96.0228,17.1388
Brisbane Lions,129.0802,112.0006,17.0796
Fremantle,119.5478,103.6711,15.8767
Geelong,118.7345,103.2766,15.4579
Carlton,128.1758,114.5109,13.6649
Sydney,122.5604,110.3751,12.1853
Gold Coast,123.9611,112.1627,11.7984
Port Adelaide,115.9391,108.5630,7.3761


202216

In [33]:
round_id = '202216'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 22.477962969050324
Overall MAE: 25.784907073814672


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Western Bulldogs,133.6158,110.6242,22.9916
Richmond,124.7133,104.7539,19.9594
Melbourne,114.1572,95.2178,18.9394
Geelong,116.1510,100.9848,15.1662
Carlton,129.3447,115.1700,14.1746
Gold Coast,131.8410,118.1067,13.7343
Collingwood,120.6362,107.9951,12.6411
Fremantle,120.0280,109.0630,10.9650
Sydney,120.5699,110.3710,10.1989
Brisbane Lions,125.0679,115.8654,9.2025


202217

In [34]:
round_id = '202217'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 32.20502810913804
Overall MAE: 26.162561252363105


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Richmond,130.8763,103.7870,27.0894
Western Bulldogs,135.1655,110.6843,24.4812
Geelong,116.2950,96.3323,19.9628
Carlton,132.5798,112.8799,19.6998
Melbourne,113.4240,96.5798,16.8442
Collingwood,123.7734,108.3968,15.3767
Gold Coast,129.2741,115.3906,13.8835
Brisbane Lions,123.3434,113.3168,10.0265
Fremantle,119.8702,110.9418,8.9285
Sydney,117.9232,111.9431,5.9801


202218

In [35]:
round_id = '202218'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 22.4882418745474
Overall MAE: 25.95843239804001


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,120.2167,93.3920,26.8246
Carlton,129.8192,110.5894,19.2299
Western Bulldogs,131.8562,113.2774,18.5788
Richmond,126.0799,107.8980,18.1820
Gold Coast,129.8653,115.4060,14.4593
Collingwood,124.0543,110.3918,13.6624
Melbourne,108.8361,98.0697,10.7664
Fremantle,122.2298,111.7497,10.4801
Sydney,119.8002,109.8547,9.9455
Brisbane Lions,121.0212,114.3193,6.7019


202219

In [36]:
round_id = '202219'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 13.404454033702088
Overall MAE: 25.297696694653805


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,116.4679,90.0844,26.3835
Western Bulldogs,129.2956,108.3587,20.9369
Richmond,126.3511,106.2739,20.0771
Collingwood,122.3393,107.8948,14.4445
Sydney,121.7845,107.8354,13.9491
Carlton,121.0804,110.5806,10.4998
Melbourne,106.9528,98.9554,7.9974
Brisbane Lions,119.1379,114.0750,5.0630
Gold Coast,127.3123,122.9146,4.3977
Port Adelaide,112.7291,111.0844,1.6447


202220

In [37]:
round_id = '202220'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 25.196197262236012
Overall MAE: 25.292621723032916


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,116.3349,86.5117,29.8232
Collingwood,122.2014,104.1304,18.0710
Richmond,122.9217,106.3149,16.6069
Western Bulldogs,127.7868,111.5141,16.2727
Sydney,123.0184,109.7056,13.3128
Carlton,119.9438,109.7168,10.2270
Essendon,116.6398,109.1043,7.5355
Brisbane Lions,119.3115,114.6494,4.6621
Gold Coast,128.3538,124.4770,3.8768
Melbourne,106.3316,105.3093,1.0223


202221

In [38]:
round_id = '202221'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 22.21307543994119
Overall MAE: 25.14597666193331


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,113.7901,84.9859,28.8042
Sydney,123.4533,99.8436,23.6097
Collingwood,121.0980,105.9685,15.1295
Essendon,121.4552,106.6501,14.8051
Richmond,121.2093,110.9207,10.2886
Western Bulldogs,120.2858,110.5052,9.7807
Gold Coast,128.6282,120.4633,8.1649
Brisbane Lions,117.7929,111.5120,6.2809
Carlton,113.4150,108.1173,5.2977
Fremantle,117.3553,116.9982,0.3571


202222

In [39]:
round_id = '202222'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 27.719704940809788
Overall MAE: 25.262964310973153


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,115.2801,83.4021,31.8780
Sydney,124.6125,98.7567,25.8558
Richmond,123.6461,108.3845,15.2616
Essendon,118.1946,106.5288,11.6659
Collingwood,118.9019,109.3265,9.5754
Carlton,114.1583,107.7275,6.4308
Brisbane Lions,118.9673,114.4172,4.5500
Melbourne,112.8132,108.8229,3.9904
Western Bulldogs,113.4917,112.4500,1.0417
Gold Coast,122.6973,123.4309,-0.7336


202223

In [40]:
round_id = '202223'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 36.502575180772396
Overall MAE: 25.751643044442684


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,119.1001,81.6611,37.4390
Sydney,127.0181,99.3928,27.6254
Richmond,123.8750,106.4059,17.4691
Brisbane Lions,117.0989,110.1377,6.9611
Melbourne,113.9466,108.1540,5.7926
Carlton,113.2505,107.5565,5.6940
Essendon,115.7912,110.4942,5.2970
Collingwood,118.2289,113.4792,4.7497
Western Bulldogs,115.1640,111.8313,3.3327
Fremantle,115.7436,114.5119,1.2317


202224

In [70]:
round_id = '2022F1'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 13.929136655589456
Overall MAE: 25.2590386115738


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,121.9372,79.4770,42.4602
Sydney,125.4900,98.3682,27.1218
Richmond,126.7123,108.4629,18.2493
Melbourne,117.2093,104.6436,12.5657
Port Adelaide,117.0048,106.7568,10.2480
Western Bulldogs,117.2852,108.9524,8.3328
Carlton,115.5793,108.9677,6.6116
Fremantle,115.7533,112.0226,3.7307
Essendon,118.4780,116.8176,1.6604
Brisbane Lions,114.2984,113.2982,1.0002


In [71]:
round_id = '2022F2'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 21.09624047367133
Overall MAE: 25.0925266860577


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,122.6046,82.0305,40.5740
Sydney,127.6244,98.0220,29.6024
Richmond,128.5454,111.9350,16.6105
Melbourne,117.3394,106.7514,10.5880
Port Adelaide,116.4528,107.0212,9.4317
Carlton,116.0267,108.0216,8.0052
Western Bulldogs,112.5740,107.7668,4.8072
Fremantle,115.7719,112.6724,3.0995
Brisbane Lions,117.4958,115.5499,1.9460
Collingwood,113.8360,116.4794,-2.6434


In [72]:
round_id = '2022F3'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.815742412462647
Overall MAE: 25.351111906304045


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,124.0978,79.4044,44.6934
Sydney,128.8550,100.4762,28.3787
Richmond,131.0876,112.2427,18.8449
Port Adelaide,113.2504,102.1386,11.1118
Carlton,112.9563,105.8353,7.1211
Melbourne,115.3892,108.3043,7.0850
Brisbane Lions,119.0488,113.5997,5.4490
Fremantle,112.4770,109.9499,2.5270
Western Bulldogs,109.0796,108.1387,0.9408
Essendon,115.8271,119.2718,-3.4447


In [73]:
round_id = '2022F4'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 75.12662863034176
Overall MAE: 27.194649562749888


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,121.6086,83.8244,37.7843
Sydney,129.3646,101.3271,28.0375
Richmond,132.7049,112.1766,20.5283
Port Adelaide,112.3588,97.6574,14.7014
Carlton,110.2789,104.6237,5.6551
Western Bulldogs,108.8735,105.2253,3.6482
Melbourne,115.4464,112.0114,3.4350
Fremantle,110.1403,108.8897,1.2505
Brisbane Lions,115.2244,115.5015,-0.2771
Essendon,116.5684,118.7206,-2.1523
